# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regresión logística: Lección 6**
### **Random Forest**

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Random Forest en este tipo de problemas. Los objetivos de este pair programming :

 - Ajustad un modelo de Random Forest a nuestros datos.


In [2]:
df = pd.read_pickle("../datos/datos-regresion-logistica/2-invistico_airline_procesado.pkl")
df.head()

,age,flight_distance,departure_delay_in_minutes,gender_oe,customer_type_oe,type_of_travel_oe,class_oe,seat_comfort_oe,departure_arrival_time_convenient_oe,food_and_drink_oe,...,inflight_entertainment_oe,online_support_oe,ease_of_online_booking_oe,on_board_service_oe,leg_room_service_oe,baggage_handling_oe,checkin_service_oe,cleanliness_oe,online_boarding_oe,satisfaction
0,-0.028305,0.301419,-0.386481,1,0,0,0,3,1,1,...,3,4,3,3,3,2,4,3,3,0
1,-1.946383,-0.272034,0.926855,1,0,1,1,1,4,1,...,1,4,4,2,2,2,5,3,4,0
2,0.236257,-1.265110,-0.386481,0,0,1,1,4,4,3,...,4,3,5,5,3,4,2,5,4,1
3,-0.821993,-0.574825,-0.386481,0,1,0,0,0,0,0,...,0,5,5,4,4,4,3,4,5,1
4,0.765382,-0.516408,-0.386481,0,0,0,1,4,4,3,...,1,4,4,4,4,3,4,4,5,1


In [3]:
# Seguimos los mismos pasos para aplicar el modelo Random Forest

X = df.drop("satisfaction", axis = 1)
y = df['satisfaction']

In [4]:
#Utilizamos los mismos hiperparámetros que nos sugería el GridSearch para el modelo Decision Tree.

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

bosque = RandomForestClassifier(max_depth=20, max_features=5, min_samples_leaf=10,
                       min_samples_split=10, random_state=42)
 
bosque.fit(x_train, y_train)

y_pred_test = bosque.predict(x_test)
y_pred_train = bosque.predict(x_train)

 - Calculad las métricas a nuestro nuevo modelo.


In [5]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    """Esta función saca las métricas para las predicciones y los datos reales.
        Args:
        - Las predicciones para el train y el test y los datos reales para el train y el test, además del modelo que vamos a utilizar.
        Returns: nos devuelve el dataframe con las métricas."""

    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)


    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [6]:
rf_results = metricas(y_test, y_pred_test, y_train, y_pred_train, "Random Forest")
rf_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.958339,0.969650,0.947568,0.958482,0.916688,test,Random Forest
1,0.969784,0.979172,0.959754,0.969366,0.939562,train,Random Forest


In [6]:
importancia_predictores = pd.DataFrame({'predictor': x_train.columns,'importancia': bosque.feature_importances_})
importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
12,inflight_entertainment_oe,0.265996
7,seat_comfort_oe,0.152987
14,ease_of_online_booking_oe,0.083082
13,online_support_oe,0.078010
15,on_board_service_oe,0.046742
4,customer_type_oe,0.044304
16,leg_room_service_oe,0.041303
9,food_and_drink_oe,0.034765
6,class_oe,0.032035
20,online_boarding_oe,0.029192


 - Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [7]:
# Visualizamos el dataframe concatenado con los dos modelos anteriores.

resultados_metricas = pd.read_csv("../datos/datos-regresion-logistica/3-invistico_airline_metricas.csv", index_col = 0)
resultados_metricas.head()

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.840837,0.844237,0.835530,0.839861,0.681670,test,Regresión logistica
1,0.843710,0.850281,0.834419,0.842275,0.687421,train,Regresión logistica
0,0.934731,0.944284,0.926031,0.935068,0.869471,test,Decision Tree gs
1,0.949781,0.958057,0.940351,0.949122,0.899554,train,Decision Tree gs


In [8]:
# Unimos el dataframe anterior (con los modelos de Regresión Logística y Desicioón Tree) al datafrme resultado del modelo Random Forest

resultados_metricas = pd.concat([resultados_metricas, rf_results], axis = 0)
resultados_metricas

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.840837,0.844237,0.835530,0.839861,0.681670,test,Regresión logistica
1,0.843710,0.850281,0.834419,0.842275,0.687421,train,Regresión logistica
0,0.934731,0.944284,0.926031,0.935068,0.869471,test,Decision Tree gs
1,0.949781,0.958057,0.940351,0.949122,0.899554,train,Decision Tree gs
0,0.958339,0.969650,0.947568,0.958482,0.916688,test,Random Forest
1,0.969784,0.979172,0.959754,0.969366,0.939562,train,Random Forest


>En este caso nos vamos a quedar con el modelo Random Forest es el que mejores resultados nos ha dado con una métrica del 0,91 en el test. Este dato que se encuentra muy cercano a 1, quiere decir que hay un alto grado de acuerdo entre el modelo y los datos de prueba, más allá del puro azar.
>
>Además, hemos acertado en un 95%, tal y cómo nos indica el accuracy; y la precisión, que en nuestro caso es la métrica que más nos interesa que sea alta (pues, considerando que tanto la aerolínea como los clientes prefieren un índice alto de satisfacción, buscamos que haya los menos falsos positivos posibles), nos indica que nuestro modelo se va a equivocar poco más de un 3% de las veces que prediga que un cliente va a estar satisfecho con el servicio. No obstante, el resto de las métricas también son altas y homogéneas.
>
>Consideramos por tanto que el modelo es robusto y fiable en sus predicciones. 


In [9]:
resultados_metricas.to_csv("../datos/datos-regresion-logistica/3-invistico_airline_metricas.csv")